In [4]:
# !pip3 install datasets

In [5]:
from datasets import load_dataset
from tqdm import tqdm
import re
import os
import json

accept = {'melayu', 'malaysia', 'malaysian', 'malay'}

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    return set(string.split())

def exist(string):
    return len(clean(string) & accept) > 0

In [6]:
dataset = load_dataset("EleutherAI/the_pile_deduplicated", split='train', streaming = True)

Resolving data files:   0%|          | 0/1650 [00:00<?, ?it/s]

In [7]:
# !rm -rf the_pile
!mkdir the_pile

In [8]:
exist('tak suka melayu')

True

In [21]:
index = 0
for d in tqdm(dataset):
    if exist(d['text']):
        filename = os.path.join('the_pile', f'{index}.json')
        with open(filename, 'w') as fopen:
            json.dump(d['text'], fopen)
        index += 1

In [12]:
!du -hs the_pile

18G	the_pile


In [14]:
from glob import glob

files = glob('the_pile/*.json')

In [16]:
with open('the-pile-filtered.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
        fopen_l.write(f'{json.dumps(data)}\n')

100%|██████████| 311031/311031 [11:42<00:00, 442.99it/s] 


In [17]:
!ls -lh the-pile-filtered.jsonl

-rw-r--r-- 1 jovyan users 17G Nov 19 15:32 the-pile-filtered.jsonl


In [19]:
from huggingface_hub import HfApi

api = HfApi()

In [20]:
api.upload_file(
    path_or_fileobj='the-pile-filtered.jsonl',
    path_in_repo="the-pile-filtered.jsonl",
    repo_id='malaysia-ai/dedup-text-dataset',
    repo_type="dataset",
)

the-pile-filtered.jsonl:   0%|          | 0.00/17.7G [00:00<?, ?B/s]

'https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/blob/main/the-pile-filtered.jsonl'